## Introduction: Map Selection

For this project, I decided to analyze OpenStreetMap data for a portion of **Los Angeles, California**. Specifically, I wanted to look a look at what is known as the Beach Cities, which is an area in Los Angeles' South Bay region. The Beach Cities include:

    1. Manhattan Beach
    2. Hermosa Beach
    3. Redondo Beach

And depending on who you're talking to, the area also includes:

    4. El Segundo
    5. Torrance
    
Torrance, California is the closest thing I have to a hometown in the US as I lived there for a good chunk of my childhood, so I was keen to take a look at it in this new, OpenStreetMap-based lens.  
    
I had to create a custom extract for the area. It can be found here on Mapzen:

- https://mapzen.com/data/metro-extracts/your-extracts/dc1c24eaf6d1

Below is a visual of the area of Los Angeles I extracted:

In [1]:
from IPython.display import Image
Image(url='http://i.imgur.com/M39vzki.png')

## Problems Encountered in the Map

Once the location was decided, I downloaded the full extract of the region and ran Python code to investigate any issues with the data. The following problems were discovered:

- **Street Names**: Incomplete ('Van Ness ___') or incorrect names ('Del Amo Blvd Apt #B'), along with street abbreviations ('Ave.' instead of 'Avenue')

- **Phone Numbers**: Inconsistent phone number formats ('(310) 217-9915' and '001 310-7927270')

- **Postal Codes**: Inconsistent postal code formats ('90277' and '90277-5906') and incorrect post codes ('CA 90260')

To tackle these issues, I had to create python scripts to clean each respective category of data. 

### Street Names

There are two things the auditing function needed to accomplish:

- Replace any abbreviations of street types with the type completely spelled out. 

        1. 'Hawthorne Blvd' -> 'Hawthorne Boulevard'
        2. 'Prospect Ave.' -> 'Prospect Avenue'
    
- Replace incorrect or incomplete street names with the corrected/complete counterparts.

        3. 'Hindry' -> 'Hindry Avenue'
        4. 'S Pacific Coast Hwy #102' -> 'S Pacific Coast Highway'

In order to discover problematic street names, I first had to use the regular expression (re) module to locate street types at the end of an address.

In [ ]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

m = street_type_re.search(street_name)

Afterwards, I matched them against a list of acceptable street types. If they weren't in the list of expected types, they would be added to a dictionary as keys, with the addresses that contain the problematic cases as the values - i.e. "{'Blvd': ('Crenshaw Blvd', 'Artesia Blvd')}. 

Having this overview allowed me to determine what my auditing function needed to accomplish. I created two dictionaries for mapping/correcting purposes - 'mapping' for regular mistakes and 'unusual' for incorrect/incomplete street names. If my function came across a problematic street type, it would refer to either of the dictionaries for the corrected version to be replaced with.

An analysis of the XML data, along with outside research on Google Maps and OpenStreetMaps, was needed to identify the missing street types for the incomplete street names.

Below is a snippet of the auditing function.

In [ ]:
#Function that cleans a given street name
def update_name(name, mapping, unusual):
    m = street_type_re.search(name) #Searches for whatever is at the end of a given address,
    #which is expected to be the street type
    
    street_type = m.group() #Allows us grab and work with the street type found
    
    if street_type not in expected: #If not one of the expected/acceptable street types, then move on towards cleaning
        
        if street_type not in mapping: #If not one of the standard abbrevation errors, 
            #then refer to 'unusual' dictionary for the replacement address
            name = unusual[street_type]
            
        else:
            corrected_street_type = mapping[street_type] #Refer to the 'mapping' dictionary for the correct street type
            name = name.replace(street_type, corrected_street_type) #Replace the abbreviated street type in the address 
            #with the correct street type
        
    return name

### Phone Numbers

I wanted a uniform format across all phone numbers. The standard I'd like to work with is the standard I usually come across for phone numbers in the US: 

- [Area Code] - [First 3 Numbers] - [Last 4 Numbers]. 
- For example, 310-212-7143. 

Example conversions:

    1. +1 310-416-9937 --> 310-416-9937
    2. 001 310-7927270 --> 310-792-7270
    
To track down any problems with the phone numbers in the dataset, I used a function that would search through the XML data for every instance of a phone number, and add it to a set of unique phone numbers. Taking a look at the set of phone numbers made it clear there were inconsistencies in formatting that the audit function would need to address.

The code for standardizing the phone format is below.

In [ ]:
#Function that reformats a given phone number
def update_phone_num(phone_num):
    
    phone_num = re.sub("[-+() ]", "", phone_num) #Removes any instance of the included characters - "-", "+", "(", ")", and 
    #blank spaces
    
    if len(phone_num) > 10: #Some numbers end up being greater than 10 characters due to country codes. This ensures we 
        #grab just the area code + number when that happens.
        phone_num = phone_num[-10:]

    return phone_num[0:3] + "-" + phone_num[3:6] + "-" + phone_num[6:] #The decided format for the numbers

### Postal Codes

The cleaning required for postal codes was straight forward as there were only a handful of problematic cases:

- Ensure the zip codes are in the 5-digit format
- To locate and extract the 5-digit zip code in cases where there are irrelevant or too much information - i.e. random white spaces or an additional four numbers after the zip code.

Example conversions:

    1. 90277-5906 --> 90277 
    2. CA 90620 --> 90620
    
To identify problems with postal codes, I used a function that would search through the XML data for every instance of a postal code, and add it to a set of unique postal codes. Again, I discovered formatting issues that needed to be addressed with the audit function.

The audit function can be found below.

In [ ]:
#Function that locates and extracts 5-digit zip codes
def update_postal_code(postalcode):
    loc = postalcode.find("90") #Outputs the index location of '90. '90' is chosen as every zip begins with '90'.
    #This allows us to locate the zip code in cases where there is other information included we wouldn't need.
    postalcode = postalcode[loc:loc+5] #Returns strictly the zip code by returning '90' + the following 3 digits
    
    return postalcode

##  Parsing the XML File and Writing the Data to a CSV

In order to begin moving our data towards a SQL database for analysis, I first had to parse the XML file and transform it from a document format to a tabular format. I was then able to write the data across multiple (5) .csv files, allowing us to easily import it to a SQL database. It is in this script where the data cleaning functions are applied.

Below is a snippet of the data. I'll specifically highlight a portion of the shape_element function, which handles the bulk of the format shaping and utilizes the audit functions I displayed above to clean the data as well. This portion focuses on shaping way_tags and nodes_tags from the XML file into a Python dictionary.

In [ ]:
# Clean and shape node or way XML element to Python dict
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):

    for tag in element.iter('tag'):
        dictag = {}
        dictag['id'] = element.attrib['id']
        if ":" in tag.attrib['k']:
            keysplit = re.split(":",tag.attrib["k"],1)
            dictag['key'] = keysplit[1]
            if dictag['key'] == 'street':
                dictag['value'] = update_name(tag.attrib['v'], mapping, unusual)
            elif dictag['key'] == 'phone':
                dictag['value'] = update_phone_num(tag.attrib['v'])
            elif dictag['key'] == 'postcode':
                dictag['value'] = update_postal_code(tag.attrib['v'])
            else:
                dictag['value'] = tag.attrib['v']
            dictag['type'] = keysplit[0]
        ...
        tags.append(dictag)

##  Importing CSV Files to SQL Database

With the CSV files written, I was able to begin the process of importing the data into a SQL database. In my case, I used SQLite 3 via my computer's command line. There are two steps I needed to accomplish to complete this task:

- Create five tables on sqlite based on a schema that matches their respective .csv files
- Import each .csv file into the appropriate  table

##  Connecting to the Database and Executing Queries Through a Python Function

In order to write and execute SQL queries in python, I had to import the sqlite3 library. Then, as I knew I would be making several queries moving forward, I created a function (*sqldata*) that allows me to establish a connection to my database (*p3.db*), execute a query, and then close the connection. I imported the pandas library and included the *read_sql_query()* function so the output from a given query will be in the form of a Pandas DataFrame.

In [50]:
import sqlite3
import pandas as pd

def sqldata(query):
    conn = sqlite3.connect('p3.db')
    data = pd.read_sql_query(query, conn)
    conn.close()
    return data

## Data Overview

In this section, I'll execute a number of SQL queries in order to analyze the dataset. 

### File Sizes 

    south-bay_california.osm ......... 306 MB
    p3.db ............................ 182 MB
    nodes.csv ........................ 127 MB
    nodes_tags.csv .................. 0.22 MB
    ways.csv ......................... 9.0 MB
    ways_tags.csv ................... 28.8 MB
    ways_nodes.cv ..................... 37 MB 

### Number of Nodes 

In [9]:
sqldata("SELECT COUNT(*) FROM nodes")

,COUNT(*)
0,1360517


### Number of Ways

In [10]:
sqldata("SELECT COUNT(*) FROM ways")

,COUNT(*)
0,131835


### Number of Unique Users

In [17]:
sqldata("SELECT COUNT(*) FROM (SELECT user FROM ways UNION SELECT user FROM nodes);")

,COUNT(*)
0,325


### Top 10 Contributing Users 

In [29]:
sqldata("SELECT user, COUNT(*) as total_contributions FROM (SELECT user FROM ways UNION ALL SELECT user FROM nodes) GROUP BY user ORDER BY total_contributions DESC LIMIT 10")

,user,total_contributions
0,Luis36995_labuildings,410843
1,dannykath_labuildings,186043
2,yurasi_import,145693
3,ridixcr_import,126491
4,schleuss_imports,115375
5,piligab_labuildings,112421
6,karitotp_labuildings,97870
7,RichRico_labuildings,83987
8,sctrojan79-import,57992
9,MtnBiker_import,17660


### Number of Users with One Contribution

In [33]:
sqldata("SELECT COUNT(*) FROM (SELECT user, COUNT(*) as total_contributions FROM (SELECT user FROM ways UNION ALL SELECT user FROM nodes) GROUP BY user HAVING total_contributions=1)")

,COUNT(*)
0,68


### Top 10 Types of Restaurants

In [66]:
sqldata("SELECT value, COUNT(*) as total FROM nodes_tags WHERE key = 'cuisine' GROUP BY value ORDER BY total DESC LIMIT 10")

,value,total
0,american,9
1,chinese,6
2,coffee_shop,5
3,mexican,5
4,pizza,5
5,burger,4
6,chicken,2
7,sandwich,2
8,Baked_Goods,1
9,"Frozen_Yogurt,_Desert,ice cream",1


### Religious Representation

In [67]:
sqldata("SELECT value, COUNT(*) as total FROM nodes_tags WHERE key = 'religion' GROUP BY value ORDER BY total DESC LIMIT 10")

,value,total
0,christian,62
1,buddhist,1
2,muslim,1


### Top 10 Type of Places

In [90]:
sqldata("SELECT value, COUNT(*) as total FROM nodes_tags WHERE key='amenity' GROUP BY value ORDER BY total DESC LIMIT 10;")

,value,total
0,school,74
1,restaurant,70
2,place_of_worship,66
3,fast_food,34
4,cafe,25
5,drinking_water,12
6,library,12
7,bank,10
8,bench,10
9,pharmacy,9


## Potential Additional Improvements  

**Increasing Submissions**

Going through this dataset, my concerns were less with the cleanliness of the data - as I found it surprisingly clean - and more with the lack of data. This part of Los Angeles is too big to have as little information as it does. I think OpenStreetMap can go a long way in developing their map database if they took on certain initiative to increase engagement with their service. 

- One possible initiative would be for OpenStreetMap to form partnerships with educational institutions such as schools, or maybe libraries, to engage students with their service. As a way to develop computer and internet literacy, computer-related courses can teach students how to use OpenStreetMap. It'll expose them to online maps, GPS technology, how to participate in open source projects, and more - all while adding data to a free resource that could benefit the members of the community and the world.


- *Anticipated Problem*: However, the concern here is that you might see an influx of dirty, unreliable data, particularly if the people behind them aren't very computer literate or only participating because it's a mandatory portion of a course. Naturally the data that come from volunteers who get involved because of their genuine passion for the project would be of higher quality.

**Ensuring Data Consistency**

- For data improvement, the biggest problem I came across my data before I cleaned it was the lack of a unified format for street types or phone numbers, or simply incomplete information. If OpenStreetMap had a hard format that street types, phone numbers, zip codes, etc. should follow - and they ensured the format is appropriate for the city/country - there would be much cleaner data for analysis.


- *Anticipated Problem*: The issue here is that you could very likely see a decrease in submissions due to the stricter guidelines. 

## Conclusion

It's clear from what we've seen that the Beach Cities OpenStretMap data is still very much incomplete. It isn't entirely surprising, as these areas do not match the size and popularity of other parts of Los Angeles - i.e. Santa Monica, Downtown LA - but there is still much in these cities to be found and explored. The upside is that a lot of the data that has been entered is fairly clean, so future OSM users who embark on the task of improving the dataset with new information won't have much to worry about with regards to cleaning prior submissions. I'm looking forward to seeing how the data for the area grows overtime, and I hope to also play a role in that mission myself. 